# Construcción de una aplicación web de recomendación de cocina

En esta lección, construirás un modelo de clasificación usando algunas de las técnicas que aprendiste en las lecciones anteriores y con el conjunto de datos de la cocina deliciosa usada a través de este serie de lecciones. Además, construirás una pequeña aplicación web para usar un modelo guardado, aprovechando el runtime web de Onnx.

Uno de los usos prácticos más útiles del aprendizaje automático es construir sistemas de recomendación, y ¡hoy puedes tomar el primer en esa dirección!

En esta lección aprenderás:

* Cómo construir un modelo y guardarlo como un modelo Onnx
* Cómo usar Netron para inspeccionar el modelo
* Cómo usar tu modelo en una aplicación web por inferencia